In [ ]:
# ===============================
# Install Dependencies
# ===============================
!pip install scikit-learn xgboost matplotlib seaborn pandas ultralytics gradio plotly

import numpy as np
import pandas as pd
import time
import gradio as gr
import plotly.graph_objects as go
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from ultralytics import YOLO

# ===============================
# 1. Create Hybrid Dataset
# ===============================
# Tabular numerical data
X_num, y = make_classification(n_samples=500, n_features=8, n_informative=5, n_classes=3, random_state=42)
df_num = pd.DataFrame(X_num, columns=[f"feature_{i}" for i in range(8)])
df_num["label"] = y

# Synthetic images (for demo, using YOLO placeholder)
# In practice, replace with actual dataset
image_paths = ["sample_img1.jpg", "sample_img2.jpg"] * 250  # Dummy placeholder
df_num["image_path"] = image_paths

# Train-test split
X_train, X_test, y_train, y_test, img_train, img_test = train_test_split(
    X_num, y, image_paths, test_size=0.3, random_state=42, stratify=y
)

# ===============================
# 2. Multi-Algorithm Evaluation
# ===============================
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(64,32), max_iter=500, random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42)
}

results = []

for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    end = time.time()

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    inf_time = end - start

    results.append([name, acc, prec, rec, f1, roc, inf_time])

results_df = pd.DataFrame(results, columns=["Model","Accuracy","Precision","Recall","F1","ROC-AUC","InferenceTime"])
print(results_df)

# ===============================
# 3. 3D Visualization (Performance Dashboard)
# ===============================
fig = go.Figure()
for i, row in results_df.iterrows():
    fig.add_trace(go.Scatter3d(
        x=[row["Accuracy"]],
        y=[row["F1"]],
        z=[row["InferenceTime"]],
        mode='markers+text',
        marker=dict(size=8),
        text=[row["Model"]],
        textposition="top center"
    ))
fig.update_layout(scene=dict(
    xaxis_title='Accuracy',
    yaxis_title='F1 Score',
    zaxis_title='Inference Time (s)'
))
fig.show()

# ===============================
# 4. YOLOv8 Real-Time 3D Detection
# ===============================
yolo_model = YOLO("yolov8n.pt")  # Small & fast model

def detect_objects_3d(image_path):
    # YOLO detection
    results = yolo_model(image_path)[0]
    annotated_img = results.plot()

    # Dummy 3D coordinates (x, y, z) for visualization
    boxes = results.boxes.xyxy.numpy()  # (x1, y1, x2, y2)
    z_coords = np.random.rand(len(boxes))  # placeholder depth
    fig3d = go.Figure()
    for i, box in enumerate(boxes):
        fig3d.add_trace(go.Scatter3d(
            x=[box[0], box[2]],
            y=[box[1], box[3]],
            z=[z_coords[i]]*2,
            mode='lines+markers',
            marker=dict(size=4),
            name=f"Object {i}"
        ))
    fig3d.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ))

    return annotated_img, fig3d

# ===============================
# 5. Gradio Interface
# ===============================
demo = gr.Interface(
    fn=detect_objects_3d,
    inputs=gr.Image(type="filepath", label="Upload Image"),
    outputs=[gr.Image(label="Annotated Image"), gr.Plot(label="3D Detected Objects")],
    title="Hybrid 3D Multi-Modal Detection + ML Dashboard",
    description="Upload an image for 3D object detection and explore ML model evaluation in 3D."
)
demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:59:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


               Model  Accuracy  Precision    Recall        F1   ROC-AUC  \
0      Random Forest  0.873333   0.880916  0.873333  0.869963  0.972733   
1                SVM  0.853333   0.855327  0.853333  0.849310  0.962667   
2  Gradient Boosting  0.866667   0.869724  0.866667  0.864408  0.962400   
3     Neural Network  0.846667   0.845223  0.846667  0.845196  0.963267   
4            XGBoost  0.826667   0.838662  0.826667  0.821320  0.957200   

   InferenceTime  
0       0.249767  
1       0.027388  
2       0.996654  
3       0.868537  
4       0.240805  


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a9c3e57a425f9b45e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
